In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
import mlflow
mlflow.set_tracking_uri(uri="http://<host>:<port>")

Churn_train_df = pd.read_csv('../Data/Raw-Data/train.csv')
Churn_validation_df = pd.read_csv('../Data/Raw-Data/test.csv')

overweight_threshold = 25
Churn_train_df['BMI'] = Churn_train_df['Weight'] / (Churn_train_df['Height'] ** 2)
Churn_train_df['Overweight'] = (Churn_train_df['BMI'] > overweight_threshold).astype(int)
Churn_train_df = Churn_train_df.drop(columns='BMI')

Churn_validation_df['BMI'] = Churn_validation_df['Weight'] / (Churn_validation_df['Height'] ** 2)
Churn_validation_df['Overweight'] = (Churn_validation_df['BMI'] > overweight_threshold).astype(int)
Churn_validation_df = Churn_validation_df.drop(columns='BMI')

columns_to_ordinal_encode=Churn_train_df.select_dtypes(exclude=[np.number]).columns
columns_to_ordinal_encode=columns_to_ordinal_encode.drop('NObeyesdad')



# Apply Ordinal Encoder to specified columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
Churn_train_df[columns_to_ordinal_encode] = ordinal_encoder.fit_transform(Churn_train_df[columns_to_ordinal_encode])

# Apply Label Encoder to target column
label_encoder = LabelEncoder()
target_column = 'NObeyesdad'
Churn_train_df[target_column] = label_encoder.fit_transform(Churn_train_df[target_column])

Churn_validation_df[columns_to_ordinal_encode] = ordinal_encoder.transform(Churn_validation_df[columns_to_ordinal_encode])


X = Churn_train_df.drop(columns=[target_column])
y = Churn_train_df[target_column]

X_val = Churn_validation_df


from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score,accuracy_score
import mlflow
from mlflow.models import infer_signature
import lightgbm as lgb
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X.drop(columns='id'), y, test_size=0.25, random_state=42)

params = {
    'objective': 'multiclass',
    'num_leaves': 10,
    'learning_rate': 0.01,
    'max_depth': -1,
    'min_child_samples': 20,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'n_estimators': 1000,
    'random_state': 42
}
# Creating XGBClassifier instance
XGBClassifier_Model = lgb.LGBMClassifier(**params)

XGBClassifier_Model.fit(X_train, y_train)

# Calculate accuracy on the test set
accuracy = XGBClassifier_Model.score(X_test, y_test)

# Predict on the test set
y_pred = XGBClassifier_Model.predict(X_test)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred,average='micro')
recall = recall_score(y_test, y_pred,average='micro')
f1 = f1_score(y_test, y_pred,average='micro')
accuracy = accuracy_score(y_test, y_pred)
classification = classification_report(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n {classification}")
print(f"Confusion Matrix:\n {confusion}")


y_val_pred = XGBClassifier_Model.predict(X_val.drop(columns='id'))

y_pred_original = pd.DataFrame(label_encoder.inverse_transform(y_val_pred),columns=['NObeyesdad'])

submission_df=pd.concat([X_val['id'],y_pred_original],axis=1)

submission_df.to_csv('submission.csv',index=False)

In [49]:
import mlflow
mlflow.set_tracking_uri(uri="http://<host>:<port>")

# Raw Data

In [28]:
Churn_train_df = pd.read_csv('../Data/Raw-Data/train.csv')
Churn_validation_df = pd.read_csv('../Data/Raw-Data/test.csv')

# Feature Engineered

In [29]:
overweight_threshold = 25
Churn_train_df['BMI'] = Churn_train_df['Weight'] / (Churn_train_df['Height'] ** 2)
Churn_train_df['Overweight'] = (Churn_train_df['BMI'] > overweight_threshold).astype(int)
Churn_train_df = Churn_train_df.drop(columns='BMI')

Churn_validation_df['BMI'] = Churn_validation_df['Weight'] / (Churn_validation_df['Height'] ** 2)
Churn_validation_df['Overweight'] = (Churn_validation_df['BMI'] > overweight_threshold).astype(int)
Churn_validation_df = Churn_validation_df.drop(columns='BMI')

In [30]:
# Churn_train_df['Total charge'] =  Churn_train_df['Total day charge'] + Churn_train_df['Total eve charge'] + Churn_train_df['Total night charge'] + Churn_train_df['Total intl charge']
# Churn_validation_df['Total charge'] = Churn_validation_df['Total day charge'] + Churn_validation_df['Total eve charge'] + Churn_validation_df['Total night charge'] + Churn_validation_df['Total intl charge']

In [31]:
# Churn_train_df = Churn_train_df.drop(columns=['Total intl charge', 'Total night charge', 'Total day charge', 'Total eve charge'])
# Churn_validation_df = Churn_validation_df.drop(columns=['Total intl charge', 'Total night charge', 'Total day charge', 'Total eve charge'])

In [32]:
Churn_train_df['NObeyesdad'].unique()

array(['Overweight_Level_II', 'Normal_Weight', 'Insufficient_Weight',
       'Obesity_Type_III', 'Obesity_Type_II', 'Overweight_Level_I',
       'Obesity_Type_I'], dtype=object)

In [33]:
import numpy as np
Churn_train_df.select_dtypes(include=[np.number]).describe().T

,count,mean,std,min,25%,50%,75%,max
id,20758.0,10378.500000,5992.462780,0.00,5189.250000,10378.500000,15567.750000,20757.000000
Age,20758.0,23.841804,5.688072,14.00,20.000000,22.815416,26.000000,61.000000
Height,20758.0,1.700245,0.087312,1.45,1.631856,1.700000,1.762887,1.975663
Weight,20758.0,87.887768,26.379443,39.00,66.000000,84.064875,111.600553,165.057269
FCVC,20758.0,2.445908,0.533218,1.00,2.000000,2.393837,3.000000,3.000000
NCP,20758.0,2.761332,0.705375,1.00,3.000000,3.000000,3.000000,4.000000
CH2O,20758.0,2.029418,0.608467,1.00,1.792022,2.000000,2.549617,3.000000
FAF,20758.0,0.981747,0.838302,0.00,0.008013,1.000000,1.587406,3.000000
TUE,20758.0,0.616756,0.602113,0.00,0.000000,0.573887,1.000000,2.000000
Overweight,20758.0,0.714134,0.451837,0.00,0.000000,1.000000,1.000000,1.000000


In [34]:
Churn_train_df.select_dtypes(exclude=[np.number]).describe().T

,count,unique,top,freq
Gender,20758,2,Female,10422
family_history_with_overweight,20758,2,yes,17014
FAVC,20758,2,yes,18982
CAEC,20758,4,Sometimes,17529
SMOKE,20758,2,no,20513
SCC,20758,2,no,20071
CALC,20758,3,Sometimes,15066
MTRANS,20758,5,Public_Transportation,16687
NObeyesdad,20758,7,Obesity_Type_III,4046


In [35]:
columns_to_ordinal_encode=Churn_train_df.select_dtypes(exclude=[np.number]).columns
columns_to_ordinal_encode=columns_to_ordinal_encode.drop('NObeyesdad')

In [36]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# Apply Ordinal Encoder to specified columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
Churn_train_df[columns_to_ordinal_encode] = ordinal_encoder.fit_transform(Churn_train_df[columns_to_ordinal_encode])

# Apply Label Encoder to target column
label_encoder = LabelEncoder()
target_column = 'NObeyesdad'
Churn_train_df[target_column] = label_encoder.fit_transform(Churn_train_df[target_column])

In [37]:
Churn_validation_df[columns_to_ordinal_encode] = ordinal_encoder.transform(Churn_validation_df[columns_to_ordinal_encode])

# Churn_validation_df[target_column] = label_encoder.transform(Churn_validation_df[target_column])

In [38]:
X = Churn_train_df.drop(columns=[target_column])
y = Churn_train_df[target_column]

X_val = Churn_validation_df
# y_val = Churn_validation_df[target_column]

In [61]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score,accuracy_score
import mlflow
from mlflow.models import infer_signature
import lightgbm as lgb
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X.drop(columns='id'), y, test_size=0.25, random_state=42)

params = {
    'objective': 'multiclass',
    'num_leaves': 10,
    'learning_rate': 0.01,
    'max_depth': -1,
    'min_child_samples': 20,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'n_estimators': 1000,
    'random_state': 42
}
# Creating XGBClassifier instance
XGBClassifier_Model = lgb.LGBMClassifier(**params)

XGBClassifier_Model.fit(X_train, y_train)

# Calculate accuracy on the test set
accuracy = XGBClassifier_Model.score(X_test, y_test)

# Predict on the test set
y_pred = XGBClassifier_Model.predict(X_test)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred,average='micro')
recall = recall_score(y_test, y_pred,average='micro')
f1 = f1_score(y_test, y_pred,average='micro')
accuracy = accuracy_score(y_test, y_pred)
classification = classification_report(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n {classification}")
print(f"Confusion Matrix:\n {confusion}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001200 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2044
[LightGBM] [Info] Number of data points in the train set: 15568, number of used features: 17
[LightGBM] [Info] Start training from score -2.112351
[LightGBM] [Info] Start training from score -1.911439
[LightGBM] [Info] Start training from score -1.945461
[LightGBM] [Info] Start training from score -1.860211
[LightGBM] [Info] Start training from score -1.636985
[LightGBM] [Info] Start training from score -2.148581
[LightGBM] [Info] Start training from score -2.108641
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

In [62]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("accuracy", accuracy)
    # mlflow.log_metric("classification", classification)
    # mlflow.log_metric("confusion", confusion)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, XGBClassifier_Model.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=XGBClassifier_Model,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

/home/aizaz/anaconda3/envs/py310/lib/python3.10/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/02/16 12:29:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation

# Predict on the test set

In [60]:
y_val_pred = XGBClassifier_Model.predict(X_val.drop(columns='id'))

y_pred_original = pd.DataFrame(label_encoder.inverse_transform(y_val_pred),columns=['NObeyesdad'])

submission_df=pd.concat([X_val['id'],y_pred_original],axis=1)

submission_df.to_csv('submission.csv',index=False)